In [1]:
import requests

url = 'https://github.com/opencv/opencv_zoo/blob/main/models/face_recognition_sface/face_recognition_sface_2021dec.onnx?raw=true'
local_path = 'face_recognition_sface_2021dec.onnx'

response = requests.get(url)
with open(local_path, 'wb') as f:
    f.write(response.content)

print(f'Model downloaded and saved as {local_path}')

Model downloaded and saved as face_recognition_sface_2021dec.onnx


In [2]:
import cv2 as cv  # Import OpenCV library for computer vision tasks
import numpy as np  # Import NumPy library for numerical operations

def visualize(input, faces, thickness=2):
    # Function to visualize detected faces and key points on an image
    if faces[1] is not None:
        # Check if any faces are detected
        coords = faces[1][0].astype(np.int32)
        # Get the coordinates of the first detected face and convert to integer
        cv.rectangle(input, (coords[0], coords[1]), (coords[0] + coords[2], coords[1] + coords[3]), (0, 255, 0), thickness)
        # Draw a rectangle around the face
        cv.circle(input, (coords[4], coords[5]), 2, (255, 0, 0), thickness)
        # Draw a circle on the first key point (e.g., left eye)
        cv.circle(input, (coords[6], coords[7]), 2, (0, 0, 255), thickness)
        # Draw a circle on the second key point (e.g., right eye)
        cv.circle(input, (coords[8], coords[9]), 2, (0, 255, 0), thickness)
        # Draw a circle on the third key point (e.g., nose)
        cv.circle(input, (coords[10], coords[11]), 2, (255, 0, 255), thickness)
        # Draw a circle on the fourth key point (e.g., left mouth corner)
        cv.circle(input, (coords[12], coords[13]), 2, (0, 255, 255), thickness)
        # Draw a circle on the fifth key point (e.g., right mouth corner)

# Initialize the face detector with a pre-trained model
detector = cv.FaceDetectorYN.create("face_detection_yunet_2023mar.onnx", "", (320, 320), 0.8, 0.3, 5000)

# Initialize the face recognizer with a pre-trained model
recognizer = cv.FaceRecognizerSF.create("face_recognition_sface_2021dec.onnx", "")

# Load the input image from a file
input_image = cv.imread('image path ...') # put the image of the person whose model showld know here
input_image_copy = input_image.copy()  # Create a copy of the input image
height, width, _ = input_image.shape  # Get the dimensions of the input image

# Set the input size for the face detector
detector.setInputSize((width, height))

# Detect faces in the input image
face1 = detector.detect(input_image)

# Visualize the detected face on the copied image
visualize(input_image_copy, face1)

# Ensure that at least one face is detected
assert face1[1] is not None, 'cannot find even an image'

# Align and crop the detected face for feature extraction
face1_crop = recognizer.alignCrop(input_image, face1[1][0])

# Extract features from the cropped face
face1_feature = recognizer.feature(face1_crop)

# Open a connection to the webcam
webcam = cv.VideoCapture(0)

# Start an infinite loop to process frames from the webcam
while True:
    ok, frame = webcam.read()  # Read a frame from the webcam
    if not ok:
        break  # Exit the loop if the frame is not read correctly
    frame = cv.flip(frame, 1)  # Flip the frame horizontally
    frame_copy = frame.copy()  # Create a copy of the frame
    frame_width, frame_height = int(webcam.get(cv.CAP_PROP_FRAME_WIDTH)), int(webcam.get(cv.CAP_PROP_FRAME_HEIGHT))
    # Get the width and height of the frame
    detector.setInputSize((frame_width, frame_height))
    # Set the input size for the face detector

    # Detect faces in the current frame
    face_frame = detector.detect(frame)
    color = (255, 0, 0)  # Set the initial color for drawing
    message = 'cannot find even a face'  # Set the initial message

    if face_frame[1] is not None:
        # Check if any faces are detected
        for face in face_frame[1]:
            visualize(frame_copy, (None, [face]))
            # Visualize the detected face on the copied frame

            # Align and crop the detected face for feature extraction
            face_frame_crop = recognizer.alignCrop(frame, face)

            # Extract features from the cropped face
            face_frame_feature = recognizer.feature(face_frame_crop)

            # Set the threshold for cosine similarity
            cosine_similarity_threshold = 0.363

            # Calculate the cosine similarity score between the reference and detected face
            cosine_score = recognizer.match(face1_feature, face_frame_feature, cv.FaceRecognizerSF_FR_COSINE)

            # Set the message and color if the desired person is not seen
            message = 'different person'
            color = (0, 0, 255)

            if cosine_score >= cosine_similarity_threshold:
                # If the similarity score is above the threshold, the desired person is seen
                message = 'same person'
                color = (0, 255, 0)

            coords = face[:-1].astype(np.int32)
            # Get the coordinates of the detected face
            p1 = coords[0], coords[1]  # Top-left corner of the face rectangle
            p2 = (coords[0] + coords[2], coords[1] + coords[3])  # Bottom-right corner of the face rectangle

            # Draw a rectangle around the detected face
            cv.rectangle(frame, p1, p2, color, 3)

            # Put the message text on the frame
            cv.putText(frame, message, (coords[0], coords[1] - 5), cv.FONT_HERSHEY_COMPLEX, 0.5, color, 2)
            # Uncomment to display cosine score
            # cv.putText(frame, str(cosine_score), (coords[0], coords[1] - 20), cv.FONT_HERSHEY_COMPLEX, 0.5, color, 2)
    else:
        # If no faces are detected, put the message text on the frame
        cv.putText(frame, message, (30, 30), cv.FONT_HERSHEY_COMPLEX, 1, color, 2)

    # Display the frame with the drawn overlays
    cv.imshow('frame', frame)
    if cv.waitKey(1) & 0xff == ord('q'):
        # Exit the loop if the 'q' key is pressed
        break

# Release the webcam and close all OpenCV windows
cv.destroyAllWindows()
webcam.release()


error: OpenCV(4.10.0) /Users/xperience/GHA-Actions-OpenCV/_work/opencv-python/opencv-python/opencv/modules/dnn/src/onnx/onnx_importer.cpp:277: error: (-5:Bad argument) Can't read ONNX file: face_detection_yunet_2023mar.onnx in function 'ONNXImporter'
